In [13]:
import json
import string

import wikipedia
import wptools
from SPARQLWrapper import SPARQLWrapper, JSON
from nltk import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize

In [3]:
catType = {"Airports": "?a a dbo:Infrastructure .", "Comics_characters": "?a a dbo:Agent .",
           "Artist": "?a a dbo:Animal .",
           "Astronauts": '?a a dbo:Animal .', "Building": "?a a dbo:Building .", "Astronomical_objects": "",
           "City": "?a a dbo:Place .", "Companies": "?a a dbo:Company .", "Foods": "?a a dbo:Food .",
           "Transport": "?a a dbo:MeanOfTransportation .", "Monuments_and_memorials": "?a a dbo:Place .",
           "Politicians": "?a a dbo:Animal .", "Sports_teams": "?a a dbo:Organisation .",
           "Sportspeople": "?a a dbo:Animal .", "Universities_and_colleges": "?a a dbo:Organisation .",
           "Written_communication": ""}
stop_words = stopwords.words('english')
porter = PorterStemmer()

In [4]:
k = 10
n = 2

## Exercise 1 :

In [27]:
data = []
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)
for cat in catType:
    SPARQL_GET_LISTS = f'''
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX dbc: <http://dbpedia.org/resource/Category:>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX dct: <http://purl.org/dc/terms/>

    SELECT DISTINCT ?page ?p ?code WHERE {{
    ?a dct:subject/skos:broader dbc:{cat} .
    {catType[cat]}
    ?a rdfs:label ?label .
    ?a foaf:name ?page .
    ?p foaf:primaryTopic ?a .
    ?a dbo:wikiPageID ?code .
    FILTER (lang(?label) = 'en')
    FILTER(!STRSTARTS(?label, "List of"))
    }}limit {k}
    '''
    sparql.setQuery(SPARQL_GET_LISTS)
    try:
        ret = sparql.queryAndConvert()
        for r in ret["results"]["bindings"]:
            url_name = r["code"]['value']  #re.findall("([^/]+$)", url)[0]
            page = wikipedia.page(pageid=url_name)
            cont = page.content
            number_of_sentences = len(sent_tokenize(cont))
            if number_of_sentences >= n:
                p = wptools.page(pageid=url_name, silent=True)
                print(page.summary)
                p.get_parse()
                p.get_wikidata()
                data.append(
                    {"name": r["page"]['value'], "url_name": url_name, "txt": page.content, "infobox": p.data["infobox"],
                     "wikidata": p.data["wikidata"],
                     "cat": cat, "url": r["p"]['value'],"wikibase":p.data["wikibase"]})
    except Exception as e:
        print()

with open("data.json", "w") as f:
    json.dump(data, f)


Nerlerit Inaat Airport (Greenlandic: Mittarfik Nerlerit Inaat, Danish: Constable Pynt Lufthavn) (IATA: CNP, ICAO: BGCO) is an airport in the Sermersooq municipality in eastern Greenland. It is located on Jameson Land and serves the town of Ittoqqortoormiit, approximately 40 km (25 mi) to the south-east. The airport can serve STOL aircraft. An AS 350 helicopter of Air Greenland is permanently housed at the airport, linking it with Ittoqqortoormiit Heliport. The helicopter also provides search and rescue capabilities within the surrounding area, and can be chartered for transport. On July 28, 2021, in Nerlerit Inaat Airport, a temperature of 23.2 °C (73.8 °F) was recorded.


Nightrunner (French: Coureur nocturne) is a fictional character created by David Hine and Kyle Higgins for publisher DC Comics. A comic book superhero, Nightrunner first appeared in Detective Comics Annual #12 (February 2011). Nightrunner is a 25-year-old French citizen of Algerian origin who lives in Clichy-sous-Boi

KeyboardInterrupt: 

## Exercise 2 :


In [39]:
file = open("data.json", "r")
data_to_process = json.load(file)

In [56]:
def preprocess(txt):
    text = txt.lower()
    text_p = "".join([char for char in text if char not in string.punctuation])
    words = word_tokenize(text_p)
    filtered_words = [word for word in words if word not in stop_words]
    stemmed = [porter.stem(word) for word in filtered_words]
    pos = pos_tag(stemmed)
    return pos, stemmed


for item in data_to_process:
    txt = preprocess(item["txt"])
    wikidata =

    break